In [1]:
import geopandas as gpd
import scipy as sp
import numpy as np
import pandas as pd
from shapely.geometry.polygon import Polygon
from libpysal.graph import Graph
from scipy.spatial import Delaunay
from scipy.sparse import csc_matrix
import pickle

In [2]:
%%time
## read in the data and swap Lon, Lat. 
## depending how the data is on the server you can delete that line
df = pd.read_csv('../data/NLD-buffer100-2.csv')
df = df.loc[:, ['Lon', 'Lat']]

CPU times: user 3.32 s, sys: 312 ms, total: 3.63 s
Wall time: 3.63 s


In [3]:
def triangle_area(tri):
    x1, y1, x2, y2, x3, y3 = tri[0][0], tri[0][1], tri[1][0], tri[1][1], tri[2][0], tri[2][1]
    return abs(0.5 * (((x2-x1)*(y3-y1))-((x3-x1)*(y2-y1))))

In [4]:
def get_cutoff_value(iteration, tri_areas):
    
    for _ in range(iteration):
        cutoff_value = tri_areas.mean()
        size = tri_areas.shape[0]
        below = (tri_areas <= cutoff_value).sum()
        above =  (tri_areas > cutoff_value).sum()

        # update for next iteration
        tri_areas = tri_areas[tri_areas <= cutoff_value]

    return size, cutoff_value, below, above

In [5]:
points = df.copy()
# points = df.iloc[:1_000_000, ]

min_buildings = 5000 ### set to 1_000, otherwise there are thousands of small triangles

iterations = 3

In [6]:
%%time
## do the triangulation
tri = Delaunay(points.values)

# get the area for every triangle
areas = [triangle_area(tri.points[x]) for x in tri.simplices]
areas = pd.Series(areas)

CPU times: user 4min 1s, sys: 5.55 s, total: 4min 7s
Wall time: 4min 7s


In [7]:
%%time

## select urban triangles that are below the cutoff
for iteration in range(1, iterations+1):
    
    total_size, cutoff_value, below, above = get_cutoff_value(iteration, areas)
    print(' number of triangles:', total_size, 
          ' cutoff:', cutoff_value,
          ' below:', below, 
          ' above:', above )
    
    urban_triangles = areas[areas<= cutoff_value]

    ## build a sparse graph to:
    ##  1. select the points within the urban triangles
    ##  2. group the points in connected components, based on touching triangles
    sources = []
    targets = []
    
    for x in tri.simplices[urban_triangles.index.values]:
        sources.append(x[0])
        targets.append(x[1])
        
        sources.append(x[0])
        targets.append(x[2])
    
        sources.append(x[1])
        targets.append(x[2])
    
    sparse_graph = csc_matrix((np.ones(len(sources)), (sources, targets)), shape=(tri.points.shape[0], tri.points.shape[0]))
    num_comps, component_labels = sp.sparse.csgraph.connected_components(sparse_graph)
    
    ## label the points: -1 - non-urban, 0 and up - indicate clusters
    vals, counts = np.unique(component_labels, return_counts=True)
    urban_cluster_labels = vals[counts > min_buildings]
    labels = pd.Series(-1, points.index)
    for uc in urban_cluster_labels:
        labels.iloc[np.where(component_labels == uc)[0]] = uc

    # save the labels, you have to change the path
    with open(f'labels_head_tails_{iteration}_{min_buildings}.pickle', 'wb') as f:
        pickle.dump(labels.values, f, protocol = pickle.HIGHEST_PROTOCOL)

 number of triangles: 62838846  cutoff: 2135.8141694395786  below: 58947829  above: 3891017
 number of triangles: 58947829  cutoff: 236.80226163707073  below: 42075668  above: 16872161
 number of triangles: 42075668  cutoff: 83.8697153964877  below: 24496352  above: 17579316
CPU times: user 1min 50s, sys: 4.89 s, total: 1min 55s
Wall time: 1min 55s


In [21]:
areas.describe().iloc[1:-1]

mean      2135.814169
std     745785.095772
min          0.000013
25%         48.497790
50%        123.411764
75%        339.802731
dtype: float64

In [10]:
iteration = 2
min_buildings = 5000

In [11]:
with open(f'labels_head_tails_{iteration}_{min_buildings}.pickle', 'rb') as f:
    labels = pickle.load(f)

In [12]:
def gen_clusters(cluster_points):

    for group in cluster_points:
        
        yield df.iloc[group, :2].values

In [13]:
%%time

# create the alpha-shapes
cluster_points = []
names = []
buildings = []
# area = []

for i, g in pd.Series(labels).groupby(labels):
    if (i == -1) or (g.index.shape[0] < min_buildings):
        continue
    names.append(i)
    cluster_points.append(g.index.values)
    buildings.append(g.shape[0])
    # areas = gdf.Area[g.index.values]
    # area.append(areas.sum())

CPU times: user 384 ms, sys: 0 ns, total: 384 ms
Wall time: 383 ms


In [14]:
from joblib import Parallel, effective_n_jobs, delayed
import libpysal

In [15]:
%%time
n_jobs = effective_n_jobs(-1)
step = 100
chunked_results = Parallel(n_jobs)(delayed(libpysal.cg.alpha_shape_auto)(cluster, step) for cluster in gen_clusters(cluster_points))
shapes = gpd.GeoDataFrame(pd.Series(chunked_results, name='geometry'))
shapes.crs = {'init' : 'epsg:3035'}
shapes['buildings'] = buildings
shapes['cluster'] = names
shapes = shapes.to_crs(epsg=4326)

CPU times: user 1.56 s, sys: 2.76 s, total: 4.32 s
Wall time: 23.8 s


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [16]:
shapes.shape

(979, 3)

In [18]:
# shapes.sort_values('buildings', ascending=False).iloc[:1000].explore()

In [17]:
# shapes.explore()

----

In [ ]:
%%time
iterations = 2


min_buildings = 4 ### set to 1_000, otherwise there are thousands of small triangles
cutoff = 'mean'

input_points = df.copy()
# input_points = df.iloc[:1_000_000, ]

In [ ]:
%%time

for i in range(iterations):

    ## get new labels
    input_labels = natural_cities_iteration(input_points, min_buildings)
    
    points_labels = pd.Series(-1, index=df.index)
    points_labels.loc[input_labels.index] = input_labels.values
    
    # save labels
    points_labels.to_csv(f'natural_cities_{i}_{cutoff}_{min_buildings}.csv')
    
    # drop non-urban points for the next iteration
    input_points = input_points[input_labels > -1]

In [ ]:
def natural_cities_iteration(points, min_buildings=4, cutoff='mean'):
    '''Label the building centroids: -1 non-urban, 0 and up indicate clusters.
    points - input data, 2d numpy array
    min_buildings - the size of a group of buildings to be considered noise, lower values make the relabeling slower. should always be > 4
    cutoff - how to compute the cutoff value, default is mean
    '''

    print(points.shape)
    
    ## do the triangulation
    tri = Delaunay(points.values)

    # get the area for every triangle
    areas = [triangle_area(tri.points[x]) for x in tri.simplices]
    areas = pd.Series(areas)

    ## select urban triangles that are below the cutoff
    if cutoff == 'mean':
        cutoff_value = areas.mean()
    else:
        cutoff_value = areas.median()

    print(cutoff_value)
    urban_triangles = areas[areas<= cutoff_value]


    ## build a sparse graph to:
    ##  1. select the points within the urban triangles
    ##  2. group the points in connected components, based on touching triangles
    sources = []
    targets = []
    
    for x in tri.simplices[urban_triangles.index.values]:
        sources.append(x[0])
        targets.append(x[1])
        
        sources.append(x[0])
        targets.append(x[2])
    
        sources.append(x[1])
        targets.append(x[2])

    sparse_graph = csc_matrix((np.ones(len(sources)), (sources, targets)), shape=(tri.points.shape[0], tri.points.shape[0]))
    num_comps, component_labels = sp.sparse.csgraph.connected_components(sparse_graph)

    ## label the points: -1 - non-urban, 0 and up - indicate clusters
    vals, counts = np.unique(component_labels, return_counts=True)
    urban_cluster_labels = vals[counts > min_buildings]
    labels = pd.Series(-1, points.index)
    for uc in urban_cluster_labels:
        labels.iloc[np.where(component_labels == uc)[0]] = uc
    return labels

In [5]:
%%time

# do the triangulation, should take around 
tri = Delaunay(points)

CPU times: user 2min 19s, sys: 5.1 s, total: 2min 24s
Wall time: 2min 24s


In [39]:
vals, counts = np.unique(component_labels, return_counts=True)

In [40]:
urban_cluster_labels = vals[counts > min_buildings]

In [41]:
labels = pd.Series(-1, gdf.index)

In [42]:
for uc in urban_cluster_labels:
    labels.iloc[np.where(component_labels == uc)[0]] = uc

In [43]:
assert labels.unique().shape[0] == urban_cluster_labels.shape[0] + 1

In [50]:
labels.unique().shape[0]

3833

In [6]:
x = tri.simplices[0]
tri.simplices.shape

(62838846, 3)

In [7]:


assert triangle_area(tri.points[x]) == Polygon(tri.points[x]).area

In [8]:
%%time
areas = [triangle_area(tri.points[x]) for x in tri.simplices]

CPU times: user 1min 38s, sys: 396 ms, total: 1min 38s
Wall time: 1min 38s


In [10]:
areas = pd.Series(areas)
areas.describe().iloc[1:-1]

mean      2135.814169
std     745785.095772
min          0.000013
25%         48.497790
50%        123.411764
75%        339.802731
dtype: float64

In [31]:
cutoff = 'median'

if cutoff == 'mean':
    cutoff_value = areas.mean()
else:
    cutoff_value = areas.median()

In [32]:
urban_triangles = areas[areas<= cutoff_value]

In [33]:
urban_triangles.index

Index([      17,       34,       37,       84,       85,      105,      109,
            198,      209,      215,
       ...
       62838836, 62838837, 62838838, 62838839, 62838840, 62838841, 62838842,
       62838843, 62838844, 62838845],
      dtype='int64', length=31419423)

In [34]:
%%time
sources = []
targets = []

for x in tri.simplices[urban_triangles.index.values]:
    sources.append(x[0])
    targets.append(x[1])
    
    sources.append(x[0])
    targets.append(x[2])

    sources.append(x[1])
    targets.append(x[2])

CPU times: user 9.11 s, sys: 1.33 s, total: 10.4 s
Wall time: 10.4 s


In [35]:
from scipy.sparse import csc_matrix

In [36]:
%%time
sparse_graph = csc_matrix((np.ones(len(sources)), (sources, targets)), shape=(tri.points.shape[0], tri.points.shape[0]))

CPU times: user 12.9 s, sys: 201 ms, total: 13.1 s
Wall time: 13 s


In [37]:
%%time
num_comps, component_labels = sp.sparse.csgraph.connected_components(sparse_graph)

CPU times: user 1.18 s, sys: 2.03 ms, total: 1.18 s
Wall time: 1.18 s


In [38]:
min_buildings = 1_000

In [39]:
vals, counts = np.unique(component_labels, return_counts=True)

In [40]:
urban_cluster_labels = vals[counts > min_buildings]

In [41]:
labels = pd.Series(-1, gdf.index)

In [42]:
for uc in urban_cluster_labels:
    labels.iloc[np.where(component_labels == uc)[0]] = uc

In [43]:
assert labels.unique().shape[0] == urban_cluster_labels.shape[0] + 1

In [50]:
labels.unique().shape[0]

3833

In [51]:
 def gen_clusters(cluster_points):
    
    for group in cluster_points:
        
        yield gdf.iloc[group, :2].values

In [52]:
%%time

# create the alpha-shapes
cluster_points = []
names = []
buildings = []
# area = []

for i, g in pd.Series(labels).groupby(labels):
    if (i == -1) or (g.index.shape[0] < min_buildings):
        continue
    names.append(i)
    cluster_points.append(g.index.values)
    buildings.append(g.shape[0])
    # areas = gdf.Area[g.index.values]
    # area.append(areas.sum())

CPU times: user 1.26 s, sys: 45.9 ms, total: 1.3 s
Wall time: 1.3 s


In [53]:
from joblib import Parallel, effective_n_jobs, delayed
import libpysal

In [54]:
%%time
n_jobs = effective_n_jobs(-1)
step = 100
chunked_results = Parallel(n_jobs)(delayed(libpysal.cg.alpha_shape_auto)(cluster, step) for cluster in gen_clusters(cluster_points))
shapes = gpd.GeoDataFrame(pd.Series(chunked_results, name='geometry'))
shapes.crs = {'init' : 'epsg:3035'}
shapes['buildings'] = buildings
shapes['cluster'] = names
shapes = shapes.to_crs(epsg=4326)

CPU times: user 1.19 s, sys: 3.7 s, total: 4.9 s
Wall time: 18.7 s


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [59]:
# shapes.explore()

In [56]:
shapes.to_file(f'natural_cities_{cutoff}_{min_buildings}.geojson', driver='GeoJSON')

In [64]:
# median_shapes.explore()

----


In [ ]:
### This is the geometry approach

In [ ]:
%%time
## do the triangulation
tri = Delaunay(points.values)

In [5]:
%%time
coord_groups = [tri.points[x] for x in tri.simplices]

UsageError: Cell magic `%%` not found.


In [ ]:
%%time
polygons = gpd.GeoSeries([Polygon(x) for x in coord_groups], name='geometry', crs={'init': 'epsg:3035'})

In [ ]:
polygons.area.describe().iloc[1:-1]

In [ ]:
cutoff = 'mean'

if cutoff == 'mean':
    cutoff_value = polygons.area.mean()
else:
    cutoff_value = polygons.area.median()

In [ ]:
urban_triangles = polygons[polygons.area <= cutoff_value]

In [ ]:
# %%time

# cannot rely on strict topology
# graph = Graph.build_fuzzy_contiguity(polygons, buffer=0.1)
# subgraph = graph.subgraph(urban_triangles.index.values)

In [ ]:
%%time
graph = Graph.build_fuzzy_contiguity(urban_triangles, buffer=0.1)

In [ ]:
# %%time
# dissolved = urban_triangles.to_frame().dissolve(subgraph.component_labels)